<a href="https://colab.research.google.com/github/KeonhoChu/KDT-/blob/main/KDT__XGBOOST_23_04_19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 노트북이 코랩에서 실행 중인지 체크합니다.
import sys
if 'google.colab' in sys.modules:
    !pip install -q --upgrade xgboost
    !wget -q https://raw.githubusercontent.com/rickiepark/handson-gb/main/Chapter06/heart_disease.csv

- target : binary value(1: 심장 질환 있는 사람 / 0:그렇지 않다)
- age : 나이
- sex : 성별(1=남성,0=여성)
- cp : 가슴 통증(chest pain) (1=전형적인 협심증, 2= 비전형적인 협심증, 3=협심증이 아닌 통증, 4=무증상)
- trestbps : 안정혈압 (입원시 mmHg)
- chol : 혈청 콜레스테롤(serum cholesterol)(mg/dl)
- fbs : 공복혈당 > 120 mg/dl (1=True, 0=False)
- restecg : 심전도 결과(0=정상, 1=ST-T파 이상)(T파 반전 및 또는 0.05mV 이상의 ST 상승 또는 감소), 2 = Estes 기준에 의해 좌심실 비대증 가능성 또는 유력
- thalach : 최대 심장 박동수
- exang : 운동으로 인한 협심증(1=yes, 2=no)
- oldpeak : 휴식 대비 운동으로 인한 ST 감소
- slope : 최대 운동 ST 세그먼트 기울기(1=상승 기울기, 2=수평,3=하강 기울기)
- ca : 형광 투시로 착색된 주요 혈관(0-3)
- thal : 탈륨 스트레스 테스트(3=정상, 6=고정 결합, 7=가역적 결함)

- verbose
    - 실행 과정 출력 여부(=0 기본값, 대규모 모델을 만들 때는 verbose=1로 선언함)

In [3]:
# 경고 끄기
import warnings
warnings.filterwarnings('ignore')

import xgboost as xgb
xgb.set_config(verbosity=0)

In [4]:
import pandas as pd
df = pd.read_csv('heart_disease.csv')
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB




``

- XGbClassifier

In [6]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [7]:
# 데이터를 X와 y로 나눕니다.
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

In [8]:
model = XGBClassifier(booster='gbtree', objective='binary:logistic')

In [9]:
# cross_val_score를 임포트합니다.
from sklearn.model_selection import cross_val_score

# numpy를 임포트합니다.
import numpy as np

# 교차 검증 점수를 구합니다.
scores = cross_val_score(model, X, y, cv=5)

# 정확도를 출력합니다.
print('정확도:', np.round(scores, 2))

# 정확도 평균을 출력합니다.
print('정확도 평균: %0.2f' % (scores.mean()))

정확도: [0.84 0.85 0.82 0.8  0.77]
정확도 평균: 0.81


- booster = 'gbtree' : 어떤 부스트 구조를 쓸지 결정하는 것. / 의사결정기반 모형(gbtree)
- objective = 'multi::logistic' / 이진 분류의 기본 손실 함수

### StratifiedKFold
- 각 fold의 타깃 레이블의 비율을 동일하게 만듦.

- 참고 : https://amueller.github.io/aml/04-model-evaluation/1-data-splitting-strategies.html

- cross_val_score()와 GridSearchCV, RandomizedSearchCV에서 분류 모델을 받으면 기본적으로 StratifiedKFold(n_splits=cv) 객체를 사용하기 때문에, cv 매개변수에 폴드 수만 지정하면 됨.

- Fold를 나누기 전에 데이터를 섞기 위해 StratifiedKFold의 shuffle 매개변수(기본값 False)를 True로 지정하여 직접 StratifiedKFold의 객체를 만들어 cv 매개변수에 전달을 함.

In [10]:
# StratifiedKFold를 임포트합니다.
from sklearn.model_selection import StratifiedKFold

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2)

In [11]:
print('전체 데이터의 클래스 분포:', np.bincount(y))
print()

for split_no, (train_idx, test_idx) in enumerate(kfold.split(X, y)):
    print('{}번째 훈련 폴드:'.format(split_no+1), np.bincount(y[train_idx]))
    print('{}번째 검증 폴드:'.format(split_no+1), np.bincount(y[test_idx]))
    print()

전체 데이터의 클래스 분포: [138 165]

1번째 훈련 폴드: [110 132]
1번째 검증 폴드: [28 33]

2번째 훈련 폴드: [110 132]
2번째 검증 폴드: [28 33]

3번째 훈련 폴드: [110 132]
3번째 검증 폴드: [28 33]

4번째 훈련 폴드: [111 132]
4번째 검증 폴드: [27 33]

5번째 훈련 폴드: [111 132]
5번째 검증 폴드: [27 33]



- 기준 모델

In [12]:
# 교차 검증 점수를 계산합니다.
scores = cross_val_score(model, X, y, cv=kfold)

# 정확도를 출력합니다.
print('정확도:', np.round(scores, 2))

# 정확도 평균을 출력합니다.
print('정확도 평균: %0.2f' % (scores.mean()))

정확도: [0.72 0.82 0.75 0.8  0.82]
정확도 평균: 0.78


- GridSearch와 RandomizedSearchCV

In [13]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2)

In [14]:
def grid_search(params, random=False): 

    xgb = XGBClassifier(booster='gbtree', objective='binary:logistic', 
                        random_state=2, verbosity=0, use_label_encoder=False)
    
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2)
    
    if random:
        grid = RandomizedSearchCV(xgb, params, cv=kfold, n_iter=20, 
                                  n_jobs=-1, random_state=2)
    else:
        grid = GridSearchCV(xgb, params, cv=kfold, n_jobs=-1)
    
    # X와 y에서 하이퍼파라미터 튜닝을 수행합니다.
    grid.fit(X, y)

    # 최상의 매개변수를 추출합니다.
    best_params = grid.best_params_

    # 최상의 매개변수를 출력합니다.
    print("최상의 매개변수:", best_params)
    
    # 최상의 점수를 추출합니다.
    best_score = grid.best_score_

    # 최상의 점수를 출력합니다.
    print("최상의 점수: {:.5f}".format(best_score))

#### Xgboost hyperparameter tuning

- n_estimators : 생성할 weak learner의 수
    - learning_rate가 낮을 땐, n_estimators를 높여야 과적합이 방지

In [15]:
grid_search(params={'n_estimators':[100, 200, 400, 800]})

최상의 매개변수: {'n_estimators': 100}
최상의 점수: 0.78235


- learning_rate['기본값':0.3] (별칭 : eta)
    - learning_rate값, learning_rate가 높을수록 과적합 하기 쉽다.-> 값을 줄이면 과대적합 방지

In [16]:
grid_search(params={'learning_rate':[0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5]})

최상의 매개변수: {'learning_rate': 0.05}
최상의 점수: 0.79585


- Max_depth
    - 트리의 깊이 / 0은 tree_method='hist', grow_policy='Lossguide'일 때 선택할 수 있으며, 깊이에 제한이 없다는 의미.
    - 값을 줄이면 과대적합을 방지.

In [17]:
grid_search(params={'max_depth':[2, 3, 5, 6, 8]})

최상의 매개변수: {'max_depth': 2}
최상의 점수: 0.79902


- gamma (별칭 : min_split_loss)
    - 값을 늘리면 과대적합을 방지
    - 보통 10보다 작은 값을 설정.

In [18]:
grid_search(params={'gamma':[0, 0.01, 0.1, 0.5, 1, 2]})

최상의 매개변수: {'gamma': 0.5}
최상의 점수: 0.79574


- min_child_weight
    - 노드 분할을 위해 필요한 최소 가중치 합.
    - 값을 늘리면 과대적합을 방지.

In [19]:
grid_search(params={'min_child_weight':[1, 2, 3, 4, 5]})

최상의 매개변수: {'min_child_weight': 5}
최상의 점수: 0.81219


- subsample
    - 부스팅 단계마다 사용할 훈련 샘플 개수의 비율
    - 값을 줄이면 과대적합을 방지.

In [20]:
grid_search(params={'subsample':[0.5, 0.7, 0.8, 0.9, 1]})

최상의 매개변수: {'subsample': 0.8}
최상의 점수: 0.79579


- colsample_bytree
    - 부스팅 단계마다 사용할 특성 개수의 비율
    - 값을 줄이면 과대적합을 방지.

In [21]:
grid_search(params={'colsample_bytree':[0.5, 0.7, 0.8, 0.9, 1]})

최상의 매개변수: {'colsample_bytree': 0.5}
최상의 점수: 0.80552


- colsample_bylevel
    - 트리 깊이마다 사용할 특성 개수의 비율
    - 값을 줄이면 과대적합을 방지.

- colsample_bynode
    - 각 노드의 분할에 사용할 특성 개수의 비율
    - 값을 줄이면 과대적합을 방지.

- scale_pos_weight
    - 불균형한 데이터에서 사용함.
    - sum(음성 샘플) / (양성 샘플) -> 데이터 불균형을 제어함.

- max_delta_step
    - 불균형이 매우 심한 데이터셋에만 권장됨.

- lambda
    - 가중치 L2 규제
    - 값을 늘리면 과대적합을 방지.
- alpha
    - 가중치 L1 규제
    - 값을 늘리면 과대적합을 방지
- missing
    - 누락된 값을 -999.0과 같은 수치로 대입함.
    - 누락된 값을 자동으로 처리해줌.

- 조기 종료 적용
    - 반복적인 머신러닝 알고리즘의 훈련 횟수를 제한함.
    - 사전에 정한 훈련 반복 횟수를 채우지 않더라도 연속적인 n번의 반복횟수 동안 모델이 향상되지 않으면 훈련을 중지함.  n은 사용자가 정함.
    - 조기 종료에서 모델이 실패할 시간을 충분히 주는 것이 중요함.
    - early_stopping_rounds=10 이면 연속적적으로 10번의 훈련 반복 동안 점수가 향상되지 않으면 훈련을 멈춤.

- eval_set과 eval_metric
    - 일반적으로 분류는 'error', 회귀는 'rmse'임.

In [22]:
# train_test_split 함수를 임포트합니다.
from sklearn.model_selection import train_test_split

# 훈련 세트와 테스트 세트로 나눕니다.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2)

In [23]:
model = XGBClassifier(booster='gbtree', objective='binary:logistic')
eval_set = [(X_test, y_test)]
eval_metric='error'
model.fit(X_train, y_train, eval_metric=eval_metric, 
          eval_set=eval_set)

# 테스트 세트에 대한 예측을 만듭니다.
y_pred = model.predict(X_test)
# 예측을 평가합니다.
accuracy = accuracy_score(y_test, y_pred)
print("정확도: %.2f%%" % (accuracy * 100.0))

[0]	validation_0-error:0.15789
[1]	validation_0-error:0.10526
[2]	validation_0-error:0.11842
[3]	validation_0-error:0.13158
[4]	validation_0-error:0.11842
[5]	validation_0-error:0.14474
[6]	validation_0-error:0.14474
[7]	validation_0-error:0.14474
[8]	validation_0-error:0.14474
[9]	validation_0-error:0.14474
[10]	validation_0-error:0.14474
[11]	validation_0-error:0.15789
[12]	validation_0-error:0.15789
[13]	validation_0-error:0.17105
[14]	validation_0-error:0.17105
[15]	validation_0-error:0.17105
[16]	validation_0-error:0.15789
[17]	validation_0-error:0.17105
[18]	validation_0-error:0.15789
[19]	validation_0-error:0.17105
[20]	validation_0-error:0.17105
[21]	validation_0-error:0.17105
[22]	validation_0-error:0.18421
[23]	validation_0-error:0.18421
[24]	validation_0-error:0.17105
[25]	validation_0-error:0.18421
[26]	validation_0-error:0.18421
[27]	validation_0-error:0.18421
[28]	validation_0-error:0.18421
[29]	validation_0-error:0.18421
[30]	validation_0-error:0.18421
[31]	validation_0-

- early_stopping_rounds

In [24]:
model = XGBClassifier(booster='gbtree', objective='binary:logistic')
eval_set = [(X_test, y_test)]
eval_metric="error"
model.fit(X_train, y_train, eval_metric=eval_metric, eval_set=eval_set, 
          early_stopping_rounds=10, verbose=True)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("정확도: %.2f%%" % (accuracy * 100.0))

[0]	validation_0-error:0.15789
[1]	validation_0-error:0.10526
[2]	validation_0-error:0.11842
[3]	validation_0-error:0.13158
[4]	validation_0-error:0.11842
[5]	validation_0-error:0.14474
[6]	validation_0-error:0.14474
[7]	validation_0-error:0.14474
[8]	validation_0-error:0.14474
[9]	validation_0-error:0.14474
[10]	validation_0-error:0.14474
[11]	validation_0-error:0.15789
정확도: 89.47%


In [25]:
model = XGBClassifier(n_estimators=5000)
eval_set = [(X_test, y_test)]
eval_metric="error"
model.fit(X_train, y_train, eval_metric=eval_metric, eval_set=eval_set, 
          early_stopping_rounds=100)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("정확도: %.2f%%" % (accuracy * 100.0))

[0]	validation_0-error:0.15789
[1]	validation_0-error:0.10526
[2]	validation_0-error:0.11842
[3]	validation_0-error:0.13158
[4]	validation_0-error:0.11842
[5]	validation_0-error:0.14474
[6]	validation_0-error:0.14474
[7]	validation_0-error:0.14474
[8]	validation_0-error:0.14474
[9]	validation_0-error:0.14474
[10]	validation_0-error:0.14474
[11]	validation_0-error:0.15789
[12]	validation_0-error:0.15789
[13]	validation_0-error:0.17105
[14]	validation_0-error:0.17105
[15]	validation_0-error:0.17105
[16]	validation_0-error:0.15789
[17]	validation_0-error:0.17105
[18]	validation_0-error:0.15789
[19]	validation_0-error:0.17105
[20]	validation_0-error:0.17105
[21]	validation_0-error:0.17105
[22]	validation_0-error:0.18421
[23]	validation_0-error:0.18421
[24]	validation_0-error:0.17105
[25]	validation_0-error:0.18421
[26]	validation_0-error:0.18421
[27]	validation_0-error:0.18421
[28]	validation_0-error:0.18421
[29]	validation_0-error:0.18421
[30]	validation_0-error:0.18421
[31]	validation_0-

- 하이퍼 파라미터의 결합.
    - 하나씩 하이퍼 파라미터 조사하기.

- n_estimators

In [27]:
grid_search(params={'n_estimators':[2, 25, 50, 75, 100]})

최상의 매개변수: {'n_estimators': 50}
최상의 점수: 0.78907


- max_depth

In [28]:
grid_search(params={'max_depth':[1, 2, 3, 4, 6, 7, 8], 
                    'n_estimators':[50]})

최상의 매개변수: {'max_depth': 1, 'n_estimators': 50}
최상의 점수: 0.83869


In [29]:
grid_search(params={'max_depth':[1, 2, 3, 4, 6, 7, 8], 
                    'n_estimators':[2, 50, 100]})

최상의 매개변수: {'max_depth': 1, 'n_estimators': 50}
최상의 점수: 0.83869


- learning_rate

In [30]:
grid_search(params={'learning_rate':[0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5], 
                    'max_depth':[1], 'n_estimators':[50]})

최상의 매개변수: {'learning_rate': 0.3, 'max_depth': 1, 'n_estimators': 50}
최상의 점수: 0.83869


- min_child_weight

In [31]:
grid_search(params={'min_child_weight':[1, 2, 3, 4, 5], 
                    'max_depth':[1], 'n_estimators':[50]})

최상의 매개변수: {'max_depth': 1, 'min_child_weight': 1, 'n_estimators': 50}
최상의 점수: 0.83869


- subsample

In [32]:
grid_search(params={'subsample':[0.5, 0.6, 0.7, 0.8, 0.9, 1],
                    'max_depth':[1], 'n_estimators':[50]})

최상의 매개변수: {'max_depth': 1, 'n_estimators': 50, 'subsample': 1}
최상의 점수: 0.83869


In [33]:
grid_search(params={'subsample':[0.5, 0.6, 0.7, 0.8, 0.9, 1], 
                    'min_child_weight':[1, 2, 3, 4, 5], 
                    'learning_rate':[0.1, 0.2, 0.3, 0.4, 0.5], 
                    'max_depth':[1, 2, 3, 4, 5], 
                    'n_estimators':[2]})

최상의 매개변수: {'learning_rate': 0.5, 'max_depth': 2, 'min_child_weight': 4, 'n_estimators': 2, 'subsample': 0.9}
최상의 점수: 0.81224


- 하이퍼 파라미터 조정

In [34]:
grid_search(params={'subsample':[0.5, 0.6, 0.7, 0.8, 0.9, 1], 
                    'min_child_weight':[1, 2, 3, 4, 5], 
                    'learning_rate':[0.1, 0.2, 0.3, 0.4, 0.5], 
                    'max_depth':[1, 2, 3, 4, 5, None], 
                    'n_estimators':[2, 25, 50, 75, 100]}, random=True)

최상의 매개변수: {'subsample': 0.6, 'n_estimators': 25, 'min_child_weight': 4, 'max_depth': 4, 'learning_rate': 0.5}
최상의 점수: 0.82208


- colsample

- colsample_bytree

In [35]:
grid_search(params={'colsample_bytree':[0.5, 0.6, 0.7, 0.8, 0.9, 1],
                    'max_depth':[1], 'n_estimators':[50]})

최상의 매개변수: {'colsample_bytree': 1, 'max_depth': 1, 'n_estimators': 50}
최상의 점수: 0.83869


- colsample_bylevel

In [36]:
grid_search(params={'colsample_bylevel':[0.5, 0.6, 0.7, 0.8, 0.9, 1],
                    'max_depth':[1], 'n_estimators':[50]})

최상의 매개변수: {'colsample_bylevel': 1, 'max_depth': 1, 'n_estimators': 50}
최상의 점수: 0.83869


- colsample_bynode

In [37]:
grid_search(params={'colsample_bynode':[0.5, 0.6, 0.7, 0.8, 0.9, 1], 
                    'colsample_bylevel':[0.5, 0.6, 0.7, 0.8, 0.9, 1], 
                    'colsample_bytree':[0.5, 0.6, 0.7, 0.8, 0.9, 1], 
                    'max_depth':[1], 'n_estimators':[50]})

최상의 매개변수: {'colsample_bylevel': 0.6, 'colsample_bynode': 0.7, 'colsample_bytree': 0.8, 'max_depth': 1, 'n_estimators': 50}
최상의 점수: 0.84852


- gamma

In [38]:
grid_search(params={'gamma':[0, 0.01, 0.05, 0.1, 0.5, 1, 2, 3], 
                    'colsample_bylevel':[0.9], 
                    'colsample_bytree':[0.8], 
                    'colsample_bynode':[0.5], 
                    'max_depth':[1], 'n_estimators':[50]})

최상의 매개변수: {'colsample_bylevel': 0.9, 'colsample_bynode': 0.5, 'colsample_bytree': 0.8, 'gamma': 0, 'max_depth': 1, 'n_estimators': 50}
최상의 점수: 0.83536
